### Interactive Choropleth Map
This notebook generates a set of interactive choropleth maps, showing various metrics of internet quality for different areas in the Province of Alberta. These maps attempt to reconcile all boundaries given in the competition data, which involves some boundaries being

In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import shapely

from folium import Choropleth, Marker
import folium

### Boundary Data

In [3]:
file_path = Path("data/Municipal_Boundaries_SHP_Geographic/")

In [20]:
rural_df = gpd.read_file(file_path/"RURAL.shp") #1 
indian_df = gpd.read_file(file_path/"INDIAN.shp") # 2
city_df = gpd.read_file(file_path/"CITY.shp") # 3
town_df = gpd.read_file(file_path/"TOWN.shp") # 4 
urbserv_df = gpd.read_file(file_path/"urbserv.shp") # 5
# hamlet_df = gpd.read_file(file_path/"HAMLETPT.shp") # points, no boundaries
village_df = gpd.read_file(file_path/"VILLAGE.shp") # 6
svillage_df = gpd.read_file(file_path/"SVILLAGE.shp") # 7 

### Internet Data

In [8]:
ookla_gdf = gpd.read_file("data/ookla/AB_ookla_data_2020.shp")

In [9]:
ookla_gdf.head(3)

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,quarter,conn_type,geometry
0,0212113033010133,807,273,84,1,1,Q1,fixed,"POLYGON ((-114.13147 53.53051, -114.12598 53.5..."
1,0212113123020220,82062,11294,19,34,4,Q1,fixed,"POLYGON ((-113.55469 53.46843, -113.54919 53.4..."
2,0212131212333013,9741,1382,24,7,2,Q1,fixed,"POLYGON ((-114.28528 51.20344, -114.27979 51.2..."


### Processing Data

Changing ookla polygons to centroids, for spatial join accuracy.

In [13]:
ookla_gdf["long"] = ookla_gdf.geometry.apply(lambda x:x.centroid.x)
ookla_gdf["lat"] = ookla_gdf.geometry.apply(lambda x:x.centroid.y)

In [14]:
ookla_temp = pd.DataFrame(ookla_gdf.drop("geometry", axis=1))
ookla_gdf = gpd.GeoDataFrame(ookla_temp, geometry=gpd.points_from_xy(
    ookla_temp.long, ookla_temp.lat))
ookla_gdf = ookla_gdf.drop(["long", "lat"], axis=1)

In [19]:
ookla_gdf.crs = {"init":"EPSG:4326"}

/home/peter/anaconda3/envs/basic-data/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
